# Modelo de LSTM para teste

## Bibliotecas

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from pathlib import Path
import os
pd.options.mode.chained_assignment = None

## Ler bases

In [3]:
# Caminho do arquivo
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base' / 'Base Ibovespa Tratada.csv'

# Carregar a base
acao = pd.read_csv(caminho)
acao['Data'] = pd.to_datetime(acao['Data'])

acao

,Data,Último,Abertura,Máxima,Mínima,Variacao (%)
0,2015-01-05,47.517,48.512,48.512,47.264,-0.0205
1,2015-01-06,48.001,47.517,48.061,47.338,0.0102
2,2015-01-07,49.463,48.006,49.882,48.006,0.0305
3,2015-01-08,49.943,49.463,50.261,49.017,0.0097
4,2015-01-09,48.840,49.955,49.955,48.501,-0.0221
...,...,...,...,...,...,...
2577,2025-05-28,138.888,139.541,139.547,138.580,-0.0047
2578,2025-05-29,138.534,138.869,139.108,137.993,-0.0025
2579,2025-05-30,137.027,138.546,138.637,136.726,-0.0109
2580,2025-06-02,136.787,137.026,138.471,136.483,-0.0017


In [4]:
# Cotação da base completa
cotacao = acao['Último'].to_numpy().reshape(-1, 1)

cotacao

array([[ 47.517],
       [ 48.001],
       [ 49.463],
       ...,
       [137.027],
       [136.787],
       [136.685]])

In [5]:
# Escalar os dados
escalador = MinMaxScaler(feature_range=(0, 1))
dados_escalados = escalador.fit_transform(cotacao)

dados_escalados

array([[0.09764845],
       [0.1023652 ],
       [0.1166129 ],
       ...,
       [0.96995507],
       [0.96761619],
       [0.96662216]])

In [6]:
# Criar conjuntos de treinamento
treinamento_x = []
treinamento_y = []

for i in range(60, len(dados_escalados)):
    treinamento_x.append(dados_escalados[i-60:i, 0])
    treinamento_y.append(dados_escalados[i, 0])

In [7]:
treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

In [8]:
# Construir o modelo
modelo = Sequential()
modelo.add(LSTM(100, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
modelo.add(LSTM(50, return_sequences=False))
modelo.add(Dense(25))
modelo.add(Dense(1))
modelo.compile(optimizer="adam", loss="mean_squared_error")

c:\Users\YUPOPIC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# Treinar o modelo
modelo.fit(treinamento_x, treinamento_y, batch_size=10, epochs=10)

Epoch 1/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - loss: 0.0276
Epoch 2/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 8.3676e-04
Epoch 3/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 7.9692e-04
Epoch 4/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 6.1701e-04
Epoch 5/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 6.5045e-04
Epoch 6/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 5.2424e-04
Epoch 7/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 5.3101e-04
Epoch 8/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 4.6297e-04
Epoch 9/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 4.2763e-04
Epoch 10/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 3.4721e-04


In [10]:
# Preparar os dados para prever o próximo dia
entrada_predicao = dados_escalados[-60:]
entrada_predicao = entrada_predicao.reshape(1, 60, 1)

In [11]:
# Prever
predicao = modelo.predict(entrada_predicao)
predicao = escalador.inverse_transform(predicao)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step


In [13]:
# Mostrar resultado
ultimo_dia = acao['Data'].max()
proximo_dia = ultimo_dia + pd.Timedelta(days=1)  # apenas incremento simples

print(f"Última data na base: {ultimo_dia.date()}")
print(f"Previsão para o próximo dia ({proximo_dia.date()}): R$ {predicao[0][0]:.3f}")


Última data na base: 2025-06-03
Previsão para o próximo dia (2025-06-04): R$ 136.624
